<a href="https://colab.research.google.com/github/ml4devs/ml4devs-notebooks/blob/master/gpt/nlp_with_gpt_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Common NLP Tasks with GPT: Sentiment Analysis, Language Translation, and Named-Entity Recognition</center></h1>

<p><center>
<address>&copy; Satish Chandra Gupta<br/>
LinkedIn: <a href="https://www.linkedin.com/in/scgupta/">scgupta</a>,
Twitter: <a href="https://twitter.com/scgupta">scgupta</a>
</address>
</center></p>

---

## Setup

### Install Pip Packages

You need Python 3.7 or higher to install [OpenAI Python API library](https://github.com/openai/openai-python).

In [1]:
# You should have Python 3.10 or higher

!python --version


Python 3.10.14


In [2]:
!pip install openai python-dotenv


### Upload `.env` File with API Keys

You can either use GPT directly from OpenAI, or you can use Azure OpenAI from Microsoft. You need to create a `.env` file and add the environment variables needed for OpenAI api.

If you are using OpenAI, check your [OpenAI account](https://platform.openai.com/api-keys) for creating API key. Your `.env` file will look like following:

```sh
$ cat .env
OPENAI_API_KEY='sk-YourOpenAiApiKeyHere'
```

If you are using Microsoft Azure OpenAI:
- Go to [Azure Portal](https://portal.azure.com/) > **All Resources**
- Filter the list with Type == Azure OpenAI
- Select the one you plan to use
- If there are none, you can [create and deploy an Azure OpenAI Service resource](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource)
- Click on **Keys and Endpoint** on the left menu
- Get `AZURE_OPENAI_API_KEY` and `AZURE_OPENAI_ENDPOINT`
- Next click **Model deployments** on the left menu, and then click **Manage Deployment** button
- Alternatively, you can go to [Azure OpenAI Studio](https://oai.azure.com/), and click **Deployments** on the left menu
- Find (the latest) API version for [Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#rest-api-versioning)

Your `.env` file will look like following:
```sh
$ cat .env
AZURE_OPENAI_API_KEY=yourAzureOpenAiApiKey
AZURE_OPENAI_ENDPOINT=https://your-azure-deployment.openai.azure.com/
AZURE_OPENAI_DEPLOYMENT_ID=your-deployment-name
AZURE_OPENAI_API_VERSION=2023-10-01-preview
```

Upload `.env` using Upload File button in Google Colab (or Jupyter Notebook). In worst case scenario, uncomment and modify the relevant lines in the following cell to create `.env` file. Please note that it is dangerous to share such notebooks or check them into git.

In [3]:
# Upload or create a .env file with (Azure) OpenAI API creds

#!echo "OPENAI_API_KEY=sk-YourOpenApiKeyHere" >> .env

#!echo "AZURE_OPENAI_API_KEY=yourAzureOpenAiApiKey" >> .env
#!echo "AZURE_OPENAI_ENDPOINT=https://your-azure-deployment.openai.azure.com/" >> .env
#!echo "AZURE_OPENAI_DEPLOYMENT_ID=your-deployment-name" >> .env
#!echo "AZURE_OPENAI_API_VERSION=2023-10-01-preview" >> .env


### Load `.env` File and Specify (Azure) OpenAI GPT Model

Load environment variables from `.env` file:

In [4]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())


Set `IS_AZURE_OPENAI` flag to `True`, if you are using Azure OpenAI:

In [5]:
IS_AZURE_OPENAI: bool = False


Specify model name:

In [6]:
from datetime import datetime

GPT35_TURBO: str = "gpt-3.5-turbo-1106" if datetime.now() < datetime(2023, 12, 11) else "gpt-3.5-turbo"


---

## Create an OpenAI Client and Specify GPT Model

In [7]:
import os
import openai


In [8]:
def create_open_ai_client():
    if IS_AZURE_OPENAI:
        return openai.AzureOpenAI(
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_ID")
        )
    else:
        return openai.OpenAI(
            api_key=os.getenv('OPENAI_API_KEY')
        )


In [9]:
openai_client = create_open_ai_client()
openai_model = os.getenv("AZURE_OPENAI_DEPLOYMENT_ID") if IS_AZURE_OPENAI else GPT35_TURBO

def get_gpt_response(prompt, model=openai_model, temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai_client.chat.completions.create(
        model=model,
        #response_format={"type": "json_object"},  # Uncomment it if your chosen model supports it
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content


In [10]:
print(get_gpt_response("Say this is test in JSON"))


{
  "message": "This is a test"
}


You are all set to use GPT for common NLP tasks such as Sentiment Analysis, Language Translation, Intent/Entity Recognition.

---

## Sentiment Analysis

Let's do sentiment analysis for food reviews. In classical ML, you will need to build a supervised classification model for sentiment analysis. You need to:

- Clean and label the data (this takes significant amount of effort)
- Divide it into train, validate, and test sets
- Preprocessing: remove stop words, stemming, etc.
- Train multiple models
- Measure inference accuracy
- Select a model, and tune its hyper-parameters
- Deploy the final model

This whole endeavour may take a couple of weeks and sometime months!

But Large Language Models (LLMs) like GPT eliminates ML model training or train it with just few examples. It is called [Zero or Few Shot Learning](https://en.wikipedia.org/wiki/Zero-shot_learning). This is because foundational LLM models are capable of doing multiple tasks.

This effectively makes many NLP capabilities accessible to developers who may not have data science and machine learning expertise. And, they can do it in few hours or days (instead of weeks and months)!

See it yourself. Here is your food review sentiment analyzer with few lines of code.

In [11]:
food_reviews = [
    "The food is great, ambience is just right, but service is slow.",
    "खाना बहुत स्वादिष्ट है, बैंगन भरता और काबुली चिकन कबाब जरूर खाएँ",
    "starters soggy and लस्सी बिलकुल पानी, बकवास खाना, waste of money",
]


In [12]:
prompt = f"""
What is the sentiment of the following review that is delimited with triple backticks?

Format your response in JSON.

Review text: ```{food_reviews[0]}```
"""

print(get_gpt_response(prompt))


{
  "sentiment": "mixed"
}


Voilà! It worked like a charm! With just 20-word long prompt! Now let's improve the prompt to get the response in a structure that you specify.

In [13]:
prompt = f"""
Identify following items from the review text that is delimited with triple backticks:
- Sentiment: (positive, mixed, or negative)
- Stars: a number rating characterizing overall sentiment, 1 star being the lowest and 5 star being the highest
- Emotions: top emotion(s), maximum 3 emotions
- Summary: human readable summary of the review and sentiments in less than 255 characters

Format your response as JSON with "sentiment", "stars", "emotions", and "summary" as the keys.

Review text: ```{food_reviews[0]}```
"""

print(get_gpt_response(prompt))


{
    "sentiment": "mixed",
    "stars": 3,
    "emotions": ["positive"],
    "summary": "Great food and ambience, but slow service."
}


---

## Language Translation

Now, let's make this sentiment analyzer multi-lingual.

GPT has language identification and translation capabilities, and you can invoke them with a simple prompt.

In [14]:
prompt = f"""
Translate the input text into English.

Format your response as JSON with values for following keys:
- text: input text as is
- language: the language of the input text
- translation: input text translated in English

Input text: ```{food_reviews[1]}```
"""

print(get_gpt_response(prompt))


{
    "text": "खाना बहुत स्वादिष्ट है, बैंगन भरता और काबुली चिकन कबाब जरूर खाएँ",
    "language": "Hindi",
    "translation": "The food is very delicious, be sure to try Baingan Bharta and Kabuli Chicken Kebab"
}


You can see the pattern:
- Breakdown the task into smaller steps
- Give specific instructions for each step
- Include input with clear demarcation
- Specify the desired structure of the output

It is almost like how you will teach a smart kid to do a specific task.

You can change the functionality by changing the prompt. You can experiment and craft an effective prompt for your NLP task.

Now let's put together sentiment analysis and translation.

In [15]:
def infer_sentiment(text):
    prompt = f"""
        Identify following items from the review text:
        - Language: language of the review text
        - Translation: review text translated in English
        - Sentiment: (positive, mixed, or negative)
        - Stars: a number rating characterizing overall sentiment, 1 star being the lowest and 5 star being the highest
        - Emotions: top emotion(s), maximum 3 emotions
        - Summary: human readable summary of the review and sentiments in less than 255 characters

        Format your response as JSON with "language", "translation", "sentiment", "stars", "emotions", and summary as the keys.

        Review text: '''{text}'''
    """

    return get_gpt_response(prompt)


In [16]:
for t in food_reviews:
    print(infer_sentiment(t))


{
    "language": "English",
    "translation": "The food is great, ambience is just right, but service is slow.",
    "sentiment": "mixed",
    "stars": 3,
    "emotions": ["satisfaction", "disappointment"],
    "summary": "Enjoyable experience with great food and ambience, but service could be improved."
}
{
    "language": "Hindi",
    "translation": "The food is very delicious, must try the stuffed brinjal and kabuli chicken kebab",
    "sentiment": "positive",
    "stars": 5,
    "emotions": ["delight", "satisfaction"],
    "summary": "Highly recommend trying the delicious stuffed brinjal and kabuli chicken kebab, extremely satisfying experience with 5-star rating"
}
{
    "language": "English",
    "translation": "starters soggy and lassi completely watery, terrible food, waste of money",
    "sentiment": "negative",
    "stars": 1,
    "emotions": ["disappointment", "disgust"],
    "summary": "Extremely disappointing experience with soggy starters, watery lassi, and terrible foo

---

## Intent/Entity Extraction

[Named-Entity Recognition (NER)](https://en.wikipedia.org/wiki/Named-entity_recognition) is another very common NLP task. For example, Chatbots and Voice Assistants have to:

- Infer what you want (intent)
- Extract the named entities from your sentences that are needed to fulfill your request
- Perform that request

For example, each of these commands to Alexa have different intent, and entities associated with it:
- Play songs by Taylor Swift
- Set an alarm for 30 minutes
- How is the weather

Let's build a multilingual intent/entity extractor for a travel assistant that can enquire, book, and cancel bus, train, and flight tickets.

In [17]:
travel_messages = [
    "I want to fly from Bangalore to Delhi",
    "मुझे कल कानपुर से लखनऊ के लिए बस टिकट बुक करना है",
    "ನನ್ನ ಬಸ್ ಟಿಕೆಟ್ ರದ್ದು ಮಾಡಿ",
]


In [18]:
prompt = f"""
Act as a travel assistant clerk. Your job is to help customers by bus, train, or flight.
Identify following items from a customer message:
- Language: language of the customer message
- Translation: customer message translated in English
- Intent: (inquire, book, or cancel)
- Mode: (bus, train, or flight)
- Date: the travel date in YYYY-MM-DD
- Source: starting place of the journey
If the information isn't present, use null as the value.

Format your response as JSON with "language", "translation", "intent", "mode", "source", and "destination".

Review test: '''{travel_messages[1]}'''
"""

print(get_gpt_response(prompt))


{
  "language": "Hindi",
  "translation": "I want to book a bus ticket from Kanpur to Lucknow for tomorrow",
  "intent": "book",
  "mode": "bus",
  "date": "2022-01-01",
  "source": "Kanpur",
  "destination": "Lucknow"
}


Well, it almost got everything right, except the date. It inferred "tomorrow" incorrectly, maybe because "today" for the model is when it was trained or deployed.

That is another important lesson: your prompt must have the needed context. Let's tell it what the date today is.

In [19]:
def travel_assistant(text):
    prompt = f"""
       Act as a travel assistant clerk. Your job is to help customers by bus, train, or flight.
       Identify following items from a customer message:
       - Language: language of the customer message
       - Translation: customer message translated in English
       - Intent: (inquire, book, or cancel)
       - Mode: (bus, train, or flight)
       - Date: the travel date in YYYY-MM-DD
       - Source: starting place of the journey
       If the information isn't present, use null as the value.

       The current date and time is {datetime.now().strftime("%d %b %Y %I:%M %p")}

       Format your response as JSON with "language", "translation", "intent", "mode", "source", and "destination".

       Review test: '''{text}'''
    """

    return get_gpt_response(prompt)


In [20]:
for t in travel_messages:
    print(travel_assistant(t))


{
    "language": "English",
    "translation": "I want to fly from Bangalore to Delhi",
    "intent": "book",
    "mode": "flight",
    "date": null,
    "source": "Bangalore",
    "destination": "Delhi"
}
{
    "language": "Hindi",
    "translation": "I want to book a bus ticket from Kanpur to Lucknow tomorrow",
    "intent": "book",
    "mode": "bus",
    "date": "2024-10-25",
    "source": "Kanpur",
    "destination": "Lucknow"
}
{
    "language": "Kannada",
    "translation": "Cancel my bus ticket",
    "intent": "cancel",
    "mode": "bus",
    "date": null,
    "source": null,
    "destination": null
}


LLMs are one of the most powerful models, and yet most accessible for developers. It reduces the time to experiment, prototype, and deploy sophisticated NLP-assisted applications.

---
<p>Copyright &copy 2023 <a href="https://www.linkedin.com/in/scgupta">Satish Chandra Gupta</a>.</p>
<img src="https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png" align="left"/> <p>&nbsp;<a href="https://creativecommons.org/licenses/by-nc-sa/4.0/">CC BY-NC-SA 4.0 International</a> License.</p>